# FLAML meets DoubleMl
With this notebook we would like to demonstrate how to use the AutoML library `FLAML` to tune learners in the DoubleML framework.

In [1]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from doubleml.datasets import make_plr_CCDDHNR2018
import doubleml as dml
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Generate synthetic data
data = make_plr_CCDDHNR2018(alpha=0.5, n_obs=1000, dim_x=50, return_type="DataFrame")

# Initialize AutoML for outcome model (ml_l)
automl_l = AutoML()
settings_l = {
    "time_budget": 120,
    "metric": 'rmse',
    "estimator_list": ['xgboost'],
    "task": 'regression',
}
automl_l.fit(X_train=data.drop(columns=["y", "d"]).values, y_train=data["y"].values, verbose=2, **settings_l)

# Initialize AutoML for treatment model (ml_m)
automl_m = AutoML()
settings_m = {
    "time_budget": 120,
    "metric": 'rmse',
    "estimator_list": ['xgboost'],
    "task": 'regression',
}
automl_m.fit(X_train=data.drop(columns=["y", "d"]).values, y_train=data["d"].values, verbose=2, **settings_m)

# Check for Overfitting: Compare in-sample (train), out-of-sample (test) MSE
# ml_m
rmse_oos_ml_m = automl_m.best_loss
rmse_oos_ml_l = automl_l.best_loss

# Create DoubleMLData object with the evaluation set
obj_dml_data = dml.DoubleMLData(data, "y", "d")

# Initialize DoubleMLPLR with the trained models from flaml
obj_dml_plr = dml.DoubleMLPLR(obj_dml_data, ml_m=automl_m.model.estimator, ml_l=automl_l.model.estimator)

# Fit the DoubleMLPLR model
obj_dml_plr.fit()

# Evaluate learners using evaluate_learners() (MSE for all nuisance components)
rmse_ins_ml_l = obj_dml_plr.evaluate_learners()['ml_l'][0]
rmse_ins_ml_m = obj_dml_plr.evaluate_learners()['ml_m'][0]

# Print results
print("Tuning RMSE (ml_m):", rmse_ins_ml_m)
print("Evaluation RMSE (ml_m):", rmse_oos_ml_m)
print("Tuning RMSE (ml_l):", rmse_ins_ml_l)
print("Evaluation RMSE (ml_l):", rmse_oos_ml_l)

Tuning RMSE (ml_m): [1.02181707]
Evaluation RMSE (ml_m): 1.020225215425113
Tuning RMSE (ml_l): [1.14851273]
Evaluation RMSE (ml_l): 1.1476134085232756
